In [1]:
# import numpy as np
import cv2
import tensorflow as tf, sys



####### av:				initialize openCV stuff 
faceDetect = cv2.CascadeClassifier('F:\Anaconda\Library\etc\haarcascades\haarcascade_frontalface_default.xml')
cam = cv2.VideoCapture(0)
ret, img = cam.read()
id = input("Enter user id : ")
#######






######## av: 				initialize tensorflow classifier

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("C:\\Users\DELL\Desktop\Face_Recognition\output_labels.txt")]

# Unpersists graph from file
with tf.gfile.FastGFile("C:\\Users\DELL\Desktop\Face_Recognition\output_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


#now all we need to do is give tensorflow the image data
##first lets create the image data and then we shall use the tensorflow code to run it on the dataset


sampleNum = 0
while (True):
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,
                                          scaleFactor=1.3,
                                          minNeighbors=5
                                          )

    for (x,y,w,h) in faces:
        sampleNum = sampleNum+1
        cv2.imwrite("dataSet/User."+str(id)+"."+str(sampleNum)+".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.waitKey(100);
        
    cv2.imshow('face',img);
    cv2.waitKey(1);
    if sampleNum > 20:
        break
cam.release()
cv2.destroyAllWindows()


#the images have been stores in dataSet, now we shall get all the images and do a classification


#todo: import images from the dataSet file




##load the images into image_path, the classifier has to run 20 times:
i = 1
while ( i <= 20):	
	#gets the images one by one	
	image_path = "dataSet/User."+str(id)+"."+str(i)+".jpg"
	image_data = tf.gfile.FastGFile(image_path, 'rb').read()
	
	with tf.Session() as sess:
    		# Feed the image_data as input to the graph and get first prediction
    		softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    		predictions = sess.run(softmax_tensor, \
             		{'DecodeJpeg/contents:0': image_data})
    
    		# Sort to show labels of first prediction in order of confidence
    		top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    
    		for node_id in top_k:
        		human_string = label_lines[node_id]
        		score = predictions[0][node_id]
        		print('%s (score = %.5f)' % (human_string, score))
	i = i +1


Enter user id :  1


NotFoundError: NewRandomAccessFile failed to Create/Open: C:\Users\DELL\Desktop\Face_Recognition\output_graph.pb : The system cannot find the file specified.
; No such file or directory

In [1]:
import face_recognition
import cv2

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
]
known_face_names = [
    "Barack Obama",
    "Joe Biden"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
import numpy as np
import cv2


faceDetect = cv2.CascadeClassifier('F:\Anaconda\Library\etc\haarcascades\haarcascade_frontalface_default.xml')
cam = cv2.VideoCapture(0)
ret, img = cam.read()
id = input("Enter user id : ")

sampleNum = 0
while (True):
    ret, img = cam.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,
                                          scaleFactor=1.3,
                                          minNeighbors=5
                                          )

    for (x,y,w,h) in faces:
        sampleNum = sampleNum+1
        cv2.imwrite("C:/Users/DELL/Desktop/Face_Recognition/dataset/User."+str(id)+"."+str(sampleNum)+".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.waitKey(100);
        
    cv2.imshow('face',img);
    cv2.waitKey(1);
    if sampleNum > 100:
        break
cam.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
import tensorflow as tf, sys



####### av:				initialize openCV stuff 
faceDetect = cv2.CascadeClassifier('F:\Anaconda\Library\etc\haarcascades\haarcascade_frontalface_default.xml')
cam = cv2.VideoCapture(0)
ret, img = cam.read()
id = input("Enter user id : ")
#######






######## av: 				initialize tensorflow classifier

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("C:\\Users\DELL\Desktop\Face_Recognition/output_labels.txt")]

# Unpersists graph from file



#now all we need to do is give tensorflow the image data
##first lets create the image data and then we shall use the tensorflow code to run it on the dataset


sampleNum = 0
while (True):
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,
                                          scaleFactor=1.3,
                                          minNeighbors=5
                                          )

    for (x,y,w,h) in faces:
        sampleNum = sampleNum+1
        cv2.imwrite("dataSet/User."+str(id)+"."+str(sampleNum)+".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.waitKey(100);
        
    cv2.imshow('face',img);
    cv2.waitKey(1);
    if sampleNum > 20:
        break
cam.release()
cv2.destroyAllWindows()


#the images have been stores in dataSet, now we shall get all the images and do a classification


#todo: import images from the dataSet file




##load the images into image_path, the classifier has to run 20 times:
i = 1
while ( i <= 20):	
	#gets the images one by one	
	image_path = "dataSet/User."+str(id)+"."+str(i)+".jpg"
	image_data = tf.gfile.FastGFile(image_path, 'rb').read()
	
	with tf.Session() as sess:
    		# Feed the image_data as input to the graph and get first prediction
    		softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    		predictions = sess.run(softmax_tensor, \
             		{'DecodeJpeg/contents:0': image_data})
    
    		# Sort to show labels of first prediction in order of confidence
    		top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    
    		for node_id in top_k:
        		human_string = label_lines[node_id]
        		score = predictions[0][node_id]
        		print('%s (score = %.5f)' % (human_string, score))
	i = i +1
